<a href="https://colab.research.google.com/github/Indresh0007/PySpark-Indresh/blob/main/ScalaProgramming_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!wget -q https://github.com/scala/scala/releases/download/v2.12.18/scala-2.12.18.deb
!tar xf spark-3.4.1-bin-hadoop3.tgz
!dpkg -i scala-2.12.18.deb

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

!curl -Lo coursier https://git.io/coursier-cli && chmod +x coursier
!./coursier launch --fork almond --scala 2.12.10 -- --install

Selecting previously unselected package scala.
(Reading database ... 126284 files and directories currently installed.)
Preparing to unpack scala-2.12.18.deb ...
Unpacking scala (2.12.18-400) ...
Setting up scala (2.12.18-400) ...
Creating system group: scala
Creating system user: scala in scala with scala daemon-user and shell /bin/false
Processing triggers for man-db (2.10.2-1) ...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 42577  100 42577    0     0  64807      0 --:--:-- --:--:-- --:--:-- 64807
Downloaded 1 missing file(s) / 58
Downloaded 2 missing file(s) / 58
Downloaded 3 missing file(s) / 58
Downloaded 4 missing file(s) / 58
Downloaded 5 missing file(s) / 58
Downloaded 6 missing file(s) / 58
Downloaded 7 m

In [2]:
!ls /usr/lib/jvm/java-11-openjdk-amd64
!ls /content/spark-3.4.1-bin-hadoop3
!ls /content/scala-2.12.18.deb
!echo $JAVA_HOME
!echo $SPARK_HOME

bin  conf  docs  include  jmods  legal	lib  man  release
bin   data	jars	    LICENSE   NOTICE  R		 RELEASE  yarn
conf  examples	kubernetes  licenses  python  README.md  sbin
/content/scala-2.12.18.deb
/usr/lib/jvm/java-11-openjdk-amd64
/content/spark-3.4.1-bin-hadoop3


In [3]:
!pip install -q findspark
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Colab Scala Spark") \
    .getOrCreate()

spark

In [10]:
%%writefile HelloWorld.scala
import org.apache.spark.sql.SparkSession
object HelloWorld {
    def main(args: Array[String]): Unit = {
        val spark = SparkSession.builder()
          .appName("ScalaAdvancedTransformations")
          .master("local[*]")
          .config("spark.driver.memory", "1g")  // Increase memory to 1 GB
          .getOrCreate()
        val sc = spark.sparkContext
        //sample data
        var rdd=sc.parallelize(Seq(
          ("apple",2),("banana",3),("apple",4),
          ("banana",1),("orange",5),("orange",6)
        ))
        //using combinedByKey (Advance transformation)
        val combined = rdd.combineByKey(
          (value: Int) => (value, 1),//CreateCombiner
          (acc: (Int, Int), value: Int) => (acc._1 + value, acc._2 + 1),//MergeValues
          (acc1: (Int, Int), acc2: (Int, Int)) => (acc1._1 + acc2._1, acc1._2 + acc2._2)//MergeCombiners
        )
        //compute average per combineByKey
        val average = combined.mapValues{ case (sum, count) => sum.toDouble / count }
        average.collect().foreach(println)
       spark.stop()
    }
}

Overwriting HelloWorld.scala


In [11]:
!scalac -classpath "$(echo /content/spark-3.4.1-bin-hadoop3/jars/*.jar | tr ' ' ':')" HelloWorld.scala

In [12]:
!scala -J-Xmx1g -classpath ".:$(echo /content/spark-3.4.1-bin-hadoop3/jars/*.jar | tr ' ' ':')" HelloWorld

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/01 04:55:09 INFO SparkContext: Running Spark version 3.4.1
25/08/01 04:55:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/01 04:55:09 INFO ResourceUtils: ==============================================================
25/08/01 04:55:09 INFO ResourceUtils: No custom resources configured for spark.driver.
25/08/01 04:55:09 INFO ResourceUtils: ==============================================================
25/08/01 04:55:09 INFO SparkContext: Submitted application: ScalaAdvancedTransformations
25/08/01 04:55:10 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
25/08/01 04:55